In [1]:
import json
import os
from openai import OpenAI # GPT-4o 사용 시
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [ ]:
import json
import os
import re
from openai import OpenAI

# --- 유틸리티 함수 ---
def sanitize_filename(name):
    """파일 이름으로 사용하기 어려운 문자를 제거하거나 대체합니다."""
    if not isinstance(name, str): # 문자열이 아닌 경우 처리
        name = str(name)
    name = re.sub(r'[<>:"/\\|?*]', '_', name) # 파일명 금지 문자 대체
    name = re.sub(r'\s+', '_', name) # 공백을 밑줄로
    return name[:100] # 파일명 길이 제한 (필요시)

# --- RequirementsLoader 클래스 ---
class RequirementsLoader:
    def load_from_file(self, filepath):
        try:
            with open(filepath, 'r', encoding='utf-8') as f:
                data = json.load(f)
            print(f"요구사항 파일 '{filepath}' 로드 성공.")
            return data
        except FileNotFoundError:
            print(f"오류: 파일 '{filepath}'를 찾을 수 없습니다.")
            return None
        except json.JSONDecodeError:
            print(f"오류: 파일 '{filepath}'가 유효한 JSON 형식이 아닙니다.")
            return None
        except Exception as e:
            print(f"파일 로드 중 예기치 않은 오류 발생: {e}")
            return None

# --- RequirementsAnalyzer 클래스 ---
class RequirementsAnalyzer:
    def __init__(self, requirements_data, openai_client=None):
        self.requirements = requirements_data
        self.client = openai_client
        self.analysis_cache = {}

    def _call_gpt(self, prompt_text, cache_key, system_message="You are a helpful AI assistant."):
        if not self.client:
            print(f"OpenAI 클라이언트가 없어 GPT 분석을 건너<0xEB><0xA9><0xB5>니다 ({cache_key}). (기본 로직 수행 또는 None 반환)")
            return None
        if cache_key in self.analysis_cache:
            return self.analysis_cache[cache_key]
        
        try:
            response = openai.ChatCompletion.create(
                model=self.model,
                messages=messages,
                temperature=0.2,
                max_tokens=2048,  # 너무 길지 않도록
            )
            result = response["choices"][0]["message"]["content"].strip()
            if not result:
                print(f"⚠️ GPT 응답이 비어 있음. 키: {cache_key}")
                return None
            return result
        except Exception as e:
            print(f"❌ GPT 호출 실패 (키: {cache_key}) → {e}")
            return None

    def get_feature_specifications(self):
        feature_specs = []
        if not self.requirements:
            print("오류: 분석할 요구사항 데이터가 없습니다.")
            return feature_specs

        target_reqs = [
            req for req in self.requirements 
            if req.get("type") == "기능적" and req.get("priority") in ["필수", "높음"]
        ]
        if not target_reqs: 
            target_reqs = [req for req in self.requirements if req.get("type") == "기능적"]

        print(f"분석 대상 기능적 요구사항 수: {len(target_reqs)}")

        for req in target_reqs[:20]: 
            req_id = req.get("id")
            description = req.get("description")
            acceptance_criteria = req.get("acceptance_criteria", "N/A")
            responsible_module = req.get("responsible_module", "미지정")
            priority = req.get("priority", "미지정")
            
            feature_specs.append({
                "id": req_id,
                "description": description,
                "acceptance_criteria": acceptance_criteria,
                "ui_suggestion_raw": f"'{description}' 기능을 구현하기 위한 UI 요소들",
                "actor_suggestion": "사용자/관리자", 
                "module": responsible_module,
                "priority": priority
            })
        
        if not feature_specs and self.requirements: 
             feature_specs.append({
                "id": "GENERAL_SYSTEM_PLACEHOLDER",
                "description": "시스템의 전반적인 기능을 나타내는 기본 페이지입니다.",
                "acceptance_criteria": "사용자는 시스템의 주요 기능에 접근할 수 있어야 합니다.",
                "ui_suggestion_raw": "메인 네비게이션, 콘텐츠 영역, 푸터",
                "actor_suggestion": "일반 사용자",
                "module": "전체 시스템",
                "priority": "필수"
             })

        print(f"{len(feature_specs)}개의 주요 기능 명세 추출(또는 준비) 완료.")
        return feature_specs

    def get_system_overview(self):
        if not self.requirements: 
            print("오류: 시스템 개요를 파악할 요구사항 데이터가 없습니다.")
            return "요구사항 데이터 없음"

        first_req_desc = self.requirements[0].get("description", "상세 설명 없음")
        num_total_reqs = len(self.requirements)
        
        sample_descriptions_for_overview = "\n".join([
            f"- ID:{req.get('id')}, 설명:{req.get('description')}" 
            for req in self.requirements[:min(10, len(self.requirements))]
        ])
        
        prompt = f"""다음은 소프트웨어 요구사항의 일부입니다:
        {sample_descriptions_for_overview}
        ---
        위 요구사항들을 종합하여, 이 시스템의 주요 목적은 무엇이며, 예상되는 주요 사용자 역할(액터)들은 누구인지, 그리고 이 시스템을 대표할 만한 간결한 이름이나 주제가 있다면 무엇인지 요약해주십시오.
        """
        overview = self._call_gpt(prompt, "system_overview_summary_v3", "You are a system architect summarizing project requirements.")
        
        if overview:
            print(f"시스템 개요 파악 (GPT): {overview[:100]}...")
            return overview
        else:
            fallback_overview = f"총 {num_total_reqs}개의 요구사항을 가진 시스템. 주요 목적은 '{first_req_desc}'와 관련될 것으로 보이며, 다양한 사용자 역할(학습자, 관리자 등)을 지원할 것으로 예상됩니다."
            print(f"시스템 개요 파악 (Fallback): {fallback_overview[:100]}...")
            return fallback_overview

# --- MockupPlanner 클래스 ---
class MockupPlanner:
    def __init__(self, feature_specs, system_overview, openai_client=None):
        self.feature_specs = feature_specs
        self.system_overview = system_overview
        self.client = openai_client
        self.analysis_cache = {}

    def _call_gpt(self, prompt_text, cache_key, system_message="You are a helpful AI assistant."):
        if not self.client:
            print(f"OpenAI 클라이언트가 없어 GPT 계획 수립을 건너<0xEB><0xA9><0xB5>니다 ({cache_key}).")
            return None
        if cache_key in self.analysis_cache:
            return self.analysis_cache[cache_key]
        
        try:
            print(f"GPT 계획 요청 중 (키: {cache_key})...")
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": prompt_text}
                ]
            )
            result = response.choices[0].message.content
            self.analysis_cache[cache_key] = result
            return result
        except Exception as e:
            print(f"GPT API 호출 중 오류 발생 ({cache_key}): {e}")
            return None

    def define_pages_and_allocate_features(self):
        if not self.feature_specs:
            print("페이지 계획을 위한 기능 명세가 없습니다.")
            return self._get_fallback_page_plan()

        features_text_for_gpt = ""
        for spec in self.feature_specs: 
            features_text_for_gpt += f"- ID: {spec['id']}\n  기능 설명: {spec['description']}\n  (UI 제안: {spec['ui_suggestion_raw']}, 대상 액터 추정: {spec['actor_suggestion']}, 우선순위: {spec.get('priority', 'N/A')})\n\n"

        prompt = f"""
        다음은 구축할 소프트웨어 시스템의 개요와 주요 기능 명세입니다:

        시스템 개요:
        {self.system_overview}

        주요 기능 명세 (ID, 설명, UI 제안, 대상 액터 추정, 우선순위 순):
        {features_text_for_gpt}

        ---
        위 정보를 바탕으로, 이 시스템에 필요한 웹 페이지(화면)들의 목록을 제안해주십시오. 
        사용자 경험 흐름(User Flow)과 정보 구조(Information Architecture)를 고려하여, 기능들이 논리적으로 그룹화되고 중복이 최소화되도록 페이지를 구성해주십시오.
        **"필수" 우선순위 기능을 반드시 포함**하는 페이지들을 우선적으로 고려해주십시오.
        시스템의 주요 목적(예: 온라인 교육 플랫폼, 스포츠 지원 포털)을 충분히 고려하여 페이지들을 제안해주십시오.

        각 페이지에 대해 다음 정보를 포함하여 **JSON 형식**으로 응답해주십시오. 
        결과는 'pages'라는 최상위 키를 가진 딕셔너리이거나, 페이지 정보 딕셔너리들의 리스트 자체일 수 있습니다. 
        만약 리스트 자체로 응답한다면, 각 요소는 다음 키들을 포함해야 합니다:
        1.  `page_name`: 페이지의 대표적인 이름 (예: "User_Login", "Learner_Dashboard", "Course_Browse_And_Apply"). 파일명으로 사용하기 좋게 영어와 밑줄로 구성해주십시오.
        2.  `page_title_ko`: 페이지의 한글 제목 (HTML title 태그 및 화면 표시용).
        3.  `page_description`: 이 페이지의 주요 목적과 핵심 기능에 대한 간략한 설명.
        4.  `target_actors`: 이 페이지를 주로 사용할 사용자 역할(들) (리스트 형태, 예: ["학습자"], ["관리자", "운영자"]).
        5.  `included_feature_ids`: 이 페이지에 포함되어야 할 주요 기능들의 ID (위 기능 명세의 ID들을 참조하여 리스트 형태로, 예: ["FUNC-001", "DATA-003"]).
        6.  `key_ui_elements_suggestion`: 이 페이지의 핵심 UI 컴포넌트들에 대한 구체적인 제안 (문자열 형태).

        만약 제안할 페이지가 없다면 빈 리스트 `[]`를 'pages' 키의 값으로 주거나, 빈 리스트 자체를 응답해주십시오.
        """

        print("GPT에 페이지 정의 및 기능 할당 요청...")
        page_definitions_str = self._call_gpt(prompt, "page_definitions_v5", # 캐시 키 변경 
                                             "You are an expert UI/UX designer and information architect. Respond ONLY in valid JSON format. The response can be a JSON object with a 'pages' key containing a list, OR it can be a list of page objects directly.")
        
        if page_definitions_str:
            try:
                match = re.search(r'```json\s*([\s\S]*?)\s*```', page_definitions_str, re.IGNORECASE)
                if match:
                    json_str_cleaned = match.group(1)
                else:
                    json_str_cleaned = page_definitions_str.strip()
                
                parsed_response = json.loads(json_str_cleaned)

                # GPT 응답 형식 유연하게 처리
                pages_list = None
                if isinstance(parsed_response, list): # 응답이 리스트 그 자체인 경우
                    pages_list = parsed_response
                    print(f"GPT로부터 {len(pages_list)}개의 페이지 계획 (리스트 직접 반환)을 성공적으로 받았습니다.")
                elif isinstance(parsed_response, dict) and "pages" in parsed_response and isinstance(parsed_response.get("pages"), list):
                    # 'pages' 키를 가진 딕셔너리인 경우
                    pages_list = parsed_response["pages"]
                    print(f"GPT로부터 {len(pages_list)}개의 페이지 계획 ('pages' 키 사용)을 성공적으로 받았습니다.")
                
                if pages_list is not None: # pages_list가 None이 아니면 (즉, 위 두 조건 중 하나라도 참이면)
                    if not pages_list: # 빈 리스트인 경우
                        print("GPT가 제안한 페이지가 없습니다. 대체 계획을 사용합니다.")
                        return self._get_fallback_page_plan()
                    return pages_list
                else: # 두 가지 유효한 형식 모두 아닌 경우
                    print(f"GPT 응답이 예상된 형식이 아닙니다. 응답 내용: {parsed_response}")
                    return self._get_fallback_page_plan()

            except json.JSONDecodeError as e:
                print(f"GPT 페이지 계획 응답 파싱 오류: {e}. 응답 내용:\n{page_definitions_str}")
                return self._get_fallback_page_plan()
            except Exception as e:
                print(f"페이지 계획 처리 중 예기치 않은 오류: {e}")
                return self._get_fallback_page_plan()
        else:
            print("GPT로부터 페이지 계획을 받지 못했습니다.")
            return self._get_fallback_page_plan()

    def _get_fallback_page_plan(self):
        print("대체 페이지 계획 사용...")
        if not self.feature_specs:
            return [{
                "page_name": "Error_No_Features",
                "page_title_ko": "오류 - 기능 정보 없음",
                "page_description": "분석할 기능 명세가 없어 페이지를 계획할 수 없습니다.",
                "target_actors": ["개발자"],
                "included_feature_ids": [],
                "key_ui_elements_suggestion": "오류 메시지 표시."
            }]
        
        main_page_features = [spec['id'] for spec in self.feature_specs if spec.get('priority') == '필수']
        if not main_page_features: 
            main_page_features = [spec['id'] for spec in self.feature_specs[:min(3, len(self.feature_specs))]]
        
        actors_for_fallback = set()
        for spec_id in main_page_features:
            spec = next((s for s in self.feature_specs if s['id'] == spec_id), None)
            if spec and spec.get('actor_suggestion'):
                current_actors = spec['actor_suggestion']
                if isinstance(current_actors, str):
                    actors_for_fallback.update(act.strip() for act in current_actors.split('/'))
                elif isinstance(current_actors, list):
                     actors_for_fallback.update(current_actors)

        return [{
            "page_name": "Main_Application_Page_Fallback",
            "page_title_ko": "주요 애플리케이션 화면 (대체)",
            "page_description": "시스템의 핵심 기능들을 제공하는 기본 페이지입니다. (GPT 계획 실패로 인한 대체 화면)",
            "target_actors": list(actors_for_fallback) if actors_for_fallback else ["일반 사용자"],
            "included_feature_ids": main_page_features,
            "key_ui_elements_suggestion": "이 페이지는 다음 기능들을 포함합니다: " + ", ".join(main_page_features) + ". 각 기능에 맞는 UI 요소(버튼, 테이블, 폼 등)가 필요합니다."
        }]

# --- HtmlGenerator 클래스 ---
class HtmlGenerator:
    def __init__(self, openai_client):
        self.client = openai_client
        self.analysis_cache = {}

    def _call_gpt(self, prompt_text, cache_key, system_message="You are a helpful AI assistant."):
        if not self.client:
            print(f"OpenAI 클라이언트가 없어 GPT HTML 생성을 건너<0xEB><0xA9><0xB5>니다 ({cache_key}).")
            return None
        if cache_key in self.analysis_cache:
            return self.analysis_cache[cache_key]
        
        try:
            print(f"GPT HTML 생성 요청 중 (키: {cache_key})...")
            response = self.client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": prompt_text}
                ]
            )
            result = response.choices[0].message.content
            self.analysis_cache[cache_key] = result
            
            match = re.search(r'```(html)?\s*([\s\S]*?)\s*```', result, re.IGNORECASE)
            if match:
                html_code = match.group(2).strip()
            else:
                html_code = result.strip()

            return html_code
        except Exception as e:
            print(f"GPT HTML 생성 API 호출 중 오류 발생 ({cache_key}): {e}")
            page_name_from_key = cache_key.replace('html_gen_v4_', '').replace('html_gen_v3_', '').replace('html_gen_v2_', '').replace('html_gen_', '')
            return f"\n<!DOCTYPE html>\n<html><head><title>오류 - {page_name_from_key}</title></head><body><h1>HTML 생성 중 오류 발생</h1><p>페이지: {page_name_from_key}</p><p>오류 내용: {e}</p></body></html>"

    def generate_html_for_page_plan(self, page_plan_details, all_feature_specs):
        if not self.client:
            page_title = page_plan_details.get('page_title_ko', '오류 페이지 (클라이언트 없음)')
            return f"\n<!DOCTYPE html>\n<html><head><title>{page_title}</title></head><body><h1>OpenAI 클라이언트가 설정되지 않았습니다.</h1><p>'{page_title}' 페이지의 HTML 목업을 생성할 수 없습니다.</p></body></html>"

        page_title_ko = page_plan_details.get("page_title_ko", "목업 페이지")
        page_name_en = page_plan_details.get("page_name", "UnknownPage")
        page_description = page_plan_details.get("page_description", "N/A")
        
        target_actors_list = page_plan_details.get("target_actors", [])
        target_actors = ", ".join(target_actors_list) if isinstance(target_actors_list, list) else str(target_actors_list)
        
        included_feature_ids = page_plan_details.get("included_feature_ids", [])
        key_ui_elements_suggestion = page_plan_details.get("key_ui_elements_suggestion", "기본 콘텐츠 영역")
        
        features_details_for_prompt = ""
        if included_feature_ids:
            for req_id in included_feature_ids:
                feature = next((spec for spec in all_feature_specs if spec["id"] == req_id), None)
                if feature:
                    features_details_for_prompt += f"- 기능 ID {feature['id']} (우선순위: {feature.get('priority', 'N/A')}): {feature['description']}\n  수용 조건: {feature.get('acceptance_criteria', 'N/A')}\n  (관련 모듈: {feature.get('module', 'N/A')})\n\n"
        else:
            features_details_for_prompt = "이 페이지에 직접적으로 할당된 세부 기능 명세가 없습니다. '페이지에 포함되어야 할 주요 기능 및 UI 요소 제안'을 중심으로 최대한 상세하게 구성해주십시오.\n"

        is_responsive = True 

        prompt = f"""
        웹 페이지의 HTML 목업 코드를 생성해주십시오.

        페이지 기본 정보:
        - 한글 페이지 제목 (HTML title 태그 및 화면 제목용): "{page_title_ko}"
        - 페이지 영문명 (내부 참조용): "{page_name_en}"
        - 페이지 주요 목적 및 사용 시나리오: "{page_description}"
        - 주요 대상 사용자: "{target_actors}"

        페이지에 포함되어야 할 주요 기능 및 UI 요소 제안 (이것을 중심으로 구성하고, 아래 세부 기능 목록을 참고하여 구체화):
        {key_ui_elements_suggestion}

        참고할 세부 기능 목록 (ID, 설명, 우선순위, 관련 모듈, 수용 조건 순):
        {features_details_for_prompt}

        다음 가이드라인을 엄격히 준수하여 작성해주십시오:
        1.  전체 HTML 문서 구조 (DOCTYPE, html, head, body 태그 포함)를 제공해주십시오.
        2.  HTML5 표준을 따르고, 시맨틱 태그(header, nav, main, section, article, aside, footer 등)를 의미에 맞게 적극적으로 사용해주십시오. 공통적으로 사용될만한 요소(예: header, nav, footer)는 일관된 형태로 포함해주십시오.
        3.  {'<meta name="viewport" content="width=device-width, initial-scale=1.0"> 태그를 <head> 안에 반드시 포함해주십시오.' if is_responsive else ''}
        4.  스타일은 최소화하고, 아주 기본적인 CSS(예: 간단한 flex/grid 레이아웃, 버튼 기본 스타일, 테이블 기본 테두리)만 HTML 코드 내 `<style>` 태그 안에 포함해주십시오. 또는 CSS 클래스명만 부여하고 (예: `<div class="container">`, `<button class="btn-primary">`) 실제 스타일은 외부 파일에서 처리한다고 가정해주십시오. **복잡한 CSS는 절대 포함하지 마십시오.**
        5.  각 주요 기능 영역이나 UI 요소에는 관련된 요구사항 ID(들)를 HTML 주석으로 명확히 달아주십시오. (관련 ID 목록: {included_feature_ids})
            형식 예시: `` 또는 ``
        6.  플레이스홀더 텍스트(예: "여기에 [내용]이 표시됩니다.")나 간단한 예시 콘텐츠(예: 테이블에 1~2개 행, 목록에 2~3개 아이템)를 포함하여 HTML 구조를 명확히 이해할 수 있도록 해주십시오.
        7.  다른 페이지로의 탐색 링크는 `href="#"` 또는 GPT가 판단하여 적절한 페이지 영문명(예: `{sanitize_filename("User_Login")}.html`)으로 처리해주십시오.
        8.  폼(form)이 필요한 경우, 관련된 입력 필드(input, select, textarea 등)와 제출 버튼(button)을 포함한 기본 구조를 만들어주십시오. 라벨(<label>)도 적절히 사용해주십시오.
        9.  테이블(table)이 필요한 경우, `<thead>`와 `<tbody>`를 포함한 기본 구조와 몇 개의 예시 행/열을 만들어주십시오. 컬럼 헤더도 명시해주십시오.

        **최종 결과물은 순수 HTML 코드만이어야 합니다. 설명이나 다른 텍스트 없이 HTML 코드 자체만 응답해주십시오.**
        """
        html_cache_key = f"html_gen_v4_{page_name_en}_{len(included_feature_ids)}_{hash(features_details_for_prompt)}"
        html_code = self._call_gpt(prompt, html_cache_key,
                                   "You are an expert HTML mockup generator. Create clean, semantic HTML based on the user's request. Ensure to include comments for related requirement IDs and their descriptions as specified. Respond ONLY with the raw HTML code, no explanations or markdown.")
        return html_code

    def save_html_to_file(self, page_name, html_content, output_dir="mockups_output_v3"):
        if not os.path.exists(output_dir):
            try:
                os.makedirs(output_dir)
            except OSError as e:
                print(f"출력 디렉토리 생성 실패 '{output_dir}': {e}")
                return

        safe_filename = sanitize_filename(page_name) + ".html"
        filepath = os.path.join(output_dir, safe_filename)
        
        try:
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(html_content)
            print(f"목업 파일 저장: {filepath}")
        except Exception as e:
            print(f"❌ GPT HTML 생성 중 예외 발생 ({cache_key}): {e}")
            import traceback
            traceback.print_exc()
            return f"""<!DOCTYPE html>
        <html><head><title>HTML 생성 실패</title></head><body>
        <h1>HTML 생성 실패</h1>
        <p>페이지 키: {cache_key}</p>
        <p>오류: {e}</p>
        </body></html>"""

# --- UiMockupAgent 클래스 (메인 에이전트) ---
class UiMockupAgent:
    def __init__(self, requirements_file_path, openai_api_key):
        self.requirements_file_path = requirements_file_path
        self.openai_client = OpenAI(api_key=openai_api_key) if openai_api_key else None

        self.loader = RequirementsLoader()
        self.requirements_data = None
        self.analyzer = None
        self.planner = None
        self.generator = None

    def run(self, output_dir="generated_mockups_final_v3"):
        print("에이전트 실행 시작...")
        # 1. 요구사항 로드
        self.requirements_data = self.loader.load_from_file(self.requirements_file_path)
        if not self.requirements_data:
            print("요구사항 로드 실패. 에이전트 실행을 중단합니다.")
            return None

        # 2. 요구사항 분석
        self.analyzer = RequirementsAnalyzer(self.requirements_data, self.openai_client)
        system_overview = self.analyzer.get_system_overview()
        feature_specs = self.analyzer.get_feature_specifications()

        if not feature_specs:
            print("기능 명세 추출 실패 또는 추출된 기능 명세가 없습니다. 에이전트 실행을 중단합니다.")
            return None

        print(f"\n시스템 개요: {system_overview}")
        print(f"추출된/준비된 주요 기능 명세 수: {len(feature_specs)}")

        # 3. 페이지 기획 (GPT 핵심 활용)
        self.planner = MockupPlanner(feature_specs, system_overview, self.openai_client)
        defined_pages_with_details = self.planner.define_pages_and_allocate_features()

        if not defined_pages_with_details or not isinstance(defined_pages_with_details, list) or not defined_pages_with_details:
            print("페이지 정의 및 기능 할당 실패. GPT 응답을 확인하거나 MockupPlanner._get_fallback_page_plan()의 결과를 확인하십시오.")
            if not defined_pages_with_details:
                print("기획된 페이지가 없습니다. 실행을 중단합니다.")
                return None

        print(f"\nGPT 또는 대체 로직으로부터 기획된 페이지 수: {len(defined_pages_with_details)}")
        for i, page_plan in enumerate(defined_pages_with_details):
            if isinstance(page_plan, dict):
                print(f"  {i+1}. 페이지 영문명: {page_plan.get('page_name')}, 한글 제목: {page_plan.get('page_title_ko')}, 관련 기능 ID 수: {len(page_plan.get('included_feature_ids', []))}")
            else:
                print(f"  {i+1}. 경고: 페이지 계획 형식이 잘못되었습니다: {page_plan}")

        # 4. HTML 목업 생성
        if not self.openai_client:
            print("OpenAI API 키가 설정되지 않아 HTML 생성을 진행할 수 없습니다.")
            return None

        self.generator = HtmlGenerator(self.openai_client)
        generated_htmls_map = {}

        for page_plan in defined_pages_with_details:
            if not isinstance(page_plan, dict):
                print(f"잘못된 페이지 계획 형식으로 HTML 생성을 건너뜁니다: {page_plan}")
                continue

            page_name_from_plan = page_plan.get("page_name")
            if not page_name_from_plan:
                page_name_from_plan = sanitize_filename(page_plan.get("page_title_ko", f"Unknown_Page_{len(generated_htmls_map) + 1}"))
                print(f"경고: page_name이 없어 page_title_ko 또는 임의 이름으로 대체합니다: {page_name_from_plan}")
                page_plan["page_name"] = page_name_from_plan

            print(f"\n'{page_name_from_plan}' HTML 생성 시도...")
            html_code = self.generator.generate_html_for_page_plan(page_plan, feature_specs)

            if html_code:
                self.generator.save_html_to_file(page_name_from_plan, html_code, output_dir)
                generated_htmls_map[page_name_from_plan] = html_code
            else:
                print(f"🔴 '{page_name_from_plan}' HTML 목업 생성 실패.")

        if generated_htmls_map:
            print("\n🟢 최종 생성된 HTML 파일 목록:")
            for idx, page_name in enumerate(generated_htmls_map.keys(), start=1):
                print(f"{idx}. {page_name}: {sanitize_filename(page_name)}.html")
        else:
            print("\n🔴 HTML 목업 생성 중 문제가 발생하여 작업이 완료되지 않았습니다.")

        return generated_htmls_map

In [4]:
if __name__ == "__main__":
    import os
    from dotenv import load_dotenv

    # .env에서 API 키 로딩
    load_dotenv()
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

    # 요구사항 파일 경로 지정
    REQUIREMENTS_FILE_PATH = "extracted_requirements_developer_focused_v5_dev_focused.json"

    # 출력 디렉터리 설정
    OUTPUT_DIR = "generated_mockups_final_v3"

    # 사전 체크
    if not OPENAI_API_KEY:
        print("❌ OpenAI API 키가 설정되지 않았습니다. .env 파일을 확인하세요.")
        exit(1)

    if not os.path.exists(REQUIREMENTS_FILE_PATH):
        print(f"❌ 요구사항 파일이 존재하지 않습니다: {REQUIREMENTS_FILE_PATH}")
        exit(1)

    # 에이전트 실행
    agent = UiMockupAgent(REQUIREMENTS_FILE_PATH, OPENAI_API_KEY)
    result = agent.run(output_dir=OUTPUT_DIR)

    # 결과 출력
    if result:
        print("\n🟢 최종 생성된 HTML 파일 목록:")
        for idx, page_name in enumerate(result.keys(), start=1):
            print(f"{idx}. {page_name}: {sanitize_filename(page_name)}.html")
    else:
        print("\n🔴 HTML 목업 생성 중 문제가 발생하여 작업이 완료되지 않았습니다.")

에이전트 실행 시작...
요구사항 파일 'extracted_requirements_developer_focused_v5_dev_focused.json' 로드 성공.
❌ GPT 호출 실패 (키: system_overview_summary_v3) → name 'openai' is not defined
시스템 개요 파악 (Fallback): 총 630개의 요구사항을 가진 시스템. 주요 목적은 '학습운영시스템은 교육 신청, 수강 이력 관리 기능을 제공해야 한다.'와 관련될 것으로 보이며, 다양한 사용자 역할(학습자, 관...
분석 대상 기능적 요구사항 수: 51
20개의 주요 기능 명세 추출(또는 준비) 완료.

시스템 개요: 총 630개의 요구사항을 가진 시스템. 주요 목적은 '학습운영시스템은 교육 신청, 수강 이력 관리 기능을 제공해야 한다.'와 관련될 것으로 보이며, 다양한 사용자 역할(학습자, 관리자 등)을 지원할 것으로 예상됩니다.
추출된/준비된 주요 기능 명세 수: 20
GPT에 페이지 정의 및 기능 할당 요청...
GPT 계획 요청 중 (키: page_definitions_v5)...
GPT로부터 8개의 페이지 계획 (리스트 직접 반환)을 성공적으로 받았습니다.

GPT 또는 대체 로직으로부터 기획된 페이지 수: 8
  1. 페이지 영문명: User_Login, 한글 제목: 사용자 로그인, 관련 기능 ID 수: 1
  2. 페이지 영문명: Learner_Dashboard, 한글 제목: 학습자 대시보드, 관련 기능 ID 수: 2
  3. 페이지 영문명: Admin_Control_Panel, 한글 제목: 관리자 제어 패널, 관련 기능 ID 수: 2
  4. 페이지 영문명: Course_Browse_And_Apply, 한글 제목: 교육 과정 조회 및 신청, 관련 기능 ID 수: 2
  5. 페이지 영문명: Content_Management, 한글 제목: 콘텐츠 관리, 관련 기능 ID 수: 1
  6. 페이지 영문명: Responsive_Web_View, 한글 

NameError: name 'result' is not defined